# Interactive Coding Session - Day 5
## Convolutional Neural Networks


Links:

* Convolution (Hyper)parameters: https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md#convolution-animations


![CNN](./cnn.jpeg)

### Basic Imports

Details on some imports:

* Conv2d: Torch layer, that implements a 2D convolution
* ReLU: Rectified Linear Unit, an activation function, given by $max(0, x)$
* MaxPool2d: Torch layer, that implements a 2D max pooling operation
* Linear: Torch layer that implements a simple weighted forward pass, $y = W * x + b$
* MSELoss: Mean Squared Error, given by $l(x, y) = mean(x_n - y_n)^2$
* Adam: Optimizer; Alternative to stochastic gradient descent
* CNNDataUtils: The data loader / utility function, made to load the dataset (stored in data.npy)

In [1]:
import torch
from torch.nn import Module, Conv2d, ReLU, MaxPool2d, Linear, MSELoss
from torch.optim import Adam

from data import CNNDataUtils

from matplotlib import pyplot as plt

### Convolutional Neural Network (CNN) class

In [2]:
class CNN(Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.fc1 = Linear(in_features=None, out_features=512)
        self.fc2 = Linear(in_features=512, out_features=2)  # out = 2, as we have two classes - CATS and NOT_CATS
        self.act = ReLU()

    def forward(self, x):
        # Conv Layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # Conv Layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # Linear / Dense Layers
        x = None
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)  # Output Layer - So, no activation

        return x

### The Network Class

In [3]:
class Network:
    def __init__(self, epochs=5, lr=1e-2):
        self.model = CNN().to('cuda')
        self.epochs = epochs
        self.learning_rate = lr
        self.optimizer = Adam(self.model.parameters(), lr=self.learning_rate)
        self.lossfn = None

        self.trainData, self.testData, self.testLabels = CNNDataUtils(batch=50).loadTorchDataset()

    def train(self):
        print(f'\nNOTE: Training model for {self.epochs} epoch(s)...')
        model = self.model
        model.train()

        for epochs in range(1, self.epochs + 1):
            for batch_idx, (data, target) in enumerate(self.trainData):
                opt = self.optimizer
                lossfn = self.lossfn
                output = model(data)

                opt.zero_grad()
                loss = lossfn(output, target)
                loss.backward()
                opt.step()

            print(f'(epoch {epochs}): completed with loss {loss.item()}')

        print('NOTE: Training complete!')

    def test(self):
        print('\nNOTE: Testing model')
        model = self.model
        td, tl = self.testData, self.testLabels
        
        correct = 0
        total = 0
        with torch.no_grad():
            for i in range(len(td)):
                real_class = torch.argmax(tl[i])
                net_out = model(td[i].view(-1, 1, 50, 50))[0]
                predicted_class = torch.argmax(net_out)

                if predicted_class == real_class:
                    correct += 1
                total += 1

        print(f'NOTE: Accuracy on the testing set is {round(correct/total * 100, 2)}%')

### Training the Model

In [8]:
net = Network(epochs=10, lr=1e-5)
print('NOTE: The model summary is given below:')
print(net.model)
net.train()
net.test()

NOTE: The model summary is given below:
CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=5184, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
  (act): ReLU()
)

NOTE: Training model for 10 epoch(s)...
(epoch 1): completed with loss 0.25656840205192566
(epoch 2): completed with loss 0.25242775678634644
(epoch 3): completed with loss 0.2325589656829834
(epoch 4): completed with loss 0.23388570547103882
(epoch 5): completed with loss 0.25606614351272583
(epoch 6): completed with loss 0.23867250978946686
(epoch 7): completed with loss 0.24607202410697937
(epoch 8): completed with loss 0.23416581749916077
(epoch 9): completed with loss 0.2443709373474121
(epoch 10): completed with loss 0.22089014947414398
NOTE: Training complete!

NOTE: Testing model
NOTE: Accura